In [5]:

import tkinter as tk
from tkinter import messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import csv
import os

# Function to calculate BMI
def calculate_bmi():
    try:
        name = name_entry.get()
        weight = float(weight_scale.get())  # Get weight from slider
        height = float(height_scale.get()) / 100  # Convert height to meters from slider
        age = int(age_entry.get())
        bmi = round(weight / (height ** 2), 2)
        bmi_label.config(text=f"{name}, your BMI: {bmi}")
        
        # Classify BMI and display health suggestions
        bmi_category = classify_bmi(bmi)
        suggestion = health_suggestions(bmi_category)
        category_label.config(text=f"Category: {bmi_category}")
        suggestion_label.config(text=f"Suggestion: {suggestion}")
        
        # Calculate daily calorie needs based on BMI and age
        daily_calories = calculate_calories(weight, height, age)
        calories_label.config(text=f"Daily Calorie Needs: {daily_calories} kcal")
        
        # Visualize BMI in the interface
        visualize_bmi(bmi)
        
        # Enable saving data
        save_button.config(state=tk.NORMAL)
        
    except ValueError:
        messagebox.showerror("Input error", "Please enter valid numeric values for weight, height, and age.")

# Function to classify BMI
def classify_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 24.9:
        return "Normal weight"
    elif 25 <= bmi < 29.9:
        return "Overweight"
    else:
        return "Obese"

# Function to provide health suggestions
def health_suggestions(bmi_category):
    if bmi_category == "Underweight":
        return "Increase your calorie intake and focus on nutrient-rich foods."
    elif bmi_category == "Normal weight":
        return "Maintain a balanced diet and regular exercise."
    elif bmi_category == "Overweight":
        return "Consider portion control and regular cardio exercises."
    else:
        return "Consult a healthcare professional and focus on regular physical activity."

# Function to calculate daily calorie needs based on weight, height, and age
def calculate_calories(weight, height, age):
    bmr = 10 * weight + 6.25 * height * 100 + 5 * age + 5  # Basic calculation for daily calorie needs (for males)
    return round(bmr, 2)

# Function to visualize BMI in the Tkinter window
def visualize_bmi(bmi):
    # Clear previous figure if it exists
    for widget in plot_frame.winfo_children():
        widget.destroy()
    
    categories = ['Underweight', 'Normal', 'Overweight', 'Obese']
    values = [18.5, 24.9, 29.9, 35]  # Upper limits of each category
    colors = ['#87CEFA', '#32CD32', '#FFA07A', '#FF6347']  # Light blue, green, orange, red
    
    # Create the figure and bar chart with reduced size
    fig = Figure(figsize=(4, 2), dpi=100)  # Reduced size
    ax = fig.add_subplot(111)
    ax.bar(categories, values, color=colors, edgecolor='black')
    
    # Highlight user's BMI
    ax.axhline(y=bmi, color='red', linestyle='--', linewidth=2, label=f'Your BMI: {bmi}')
    
    # Set chart aesthetics with smaller font sizes
    ax.set_title('BMI Comparison', fontsize=12, fontweight='bold')
    ax.set_xlabel('Categories', fontsize=10)
    ax.set_ylabel('BMI Value', fontsize=10)
    ax.legend(fontsize=8, loc='upper left')
    
    # Create canvas to display the chart in the Tkinter window
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Function to reset all fields and labels
def reset_fields():
    name_entry.delete(0, tk.END)
    weight_scale.set(60)  # Reset to default value
    height_scale.set(170)  # Reset to default value
    age_entry.delete(0, tk.END)
    bmi_label.config(text="Your BMI: ")
    category_label.config(text="Category: ")
    suggestion_label.config(text="Suggestion: ")
    calories_label.config(text="Daily Calorie Needs: ")
    save_button.config(state=tk.DISABLED)
    for widget in plot_frame.winfo_children():
        widget.destroy()

# Function to save data to a CSV file
def save_data():
    name = name_entry.get()
    weight = weight_scale.get()
    height = height_scale.get()
    age = age_entry.get()
    bmi = bmi_label.cget("text").split(": ")[1]
    category = category_label.cget("text").split(": ")[1]
    suggestion = suggestion_label.cget("text").split(": ")[1]
    calories = calories_label.cget("text").split(": ")[1]

    data = [name, weight, height, age, bmi, category, suggestion, calories]

    # Check if file exists, if not, write the header first
    file_exists = os.path.isfile('bmi_data.csv')

    with open('bmi_data.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        
        if not file_exists:
            writer.writerow(['Name', 'Weight (kg)', 'Height (cm)', 'Age', 'BMI', 'Category', 'Suggestion', 'Calories (kcal)'])  # Header
        
        writer.writerow(data)
        messagebox.showinfo("Data Saved", "Your data has been saved successfully!")

# Create main application window
root = tk.Tk()
root.title("Enhanced BMI Calculator")
root.geometry("450x650")  # Adjusted window size
root.config(bg="lightgreen")  # Set background color

# Create a frame to hold the input fields and buttons
frame = tk.Frame(root, bg="#D3D3D3", padx=20, pady=20, relief=tk.RAISED, bd=2)
frame.grid(row=0, column=0, padx=20, pady=20)

# Create labels and entry fields for user input
tk.Label(frame, text="Enter your name:", font=("Arial", 12), bg="#D3D3D3").grid(row=0, column=0, padx=10, pady=10)
name_entry = tk.Entry(frame, font=("Arial", 12))
name_entry.grid(row=0, column=1, padx=10, pady=10)

tk.Label(frame, text="Enter your age:", font=("Arial", 12), bg="#D3D3D3").grid(row=1, column=0, padx=10, pady=10)
age_entry = tk.Entry(frame, font=("Arial", 12))
age_entry.grid(row=1, column=1, padx=10, pady=10)

# Add sliders for weight and height
tk.Label(frame, text="Select your weight (kg):", font=("Arial", 12), bg="#D3D3D3").grid(row=2, column=0, padx=10, pady=10)
weight_scale = tk.Scale(frame, from_=30, to=150, orient=tk.HORIZONTAL, length=200, font=("Arial", 10), tickinterval=20)
weight_scale.set(60)  # Default value
weight_scale.grid(row=2, column=1, padx=10, pady=10)

tk.Label(frame, text="Select your height (cm):", font=("Arial", 12), bg="#D3D3D3").grid(row=3, column=0, padx=10, pady=10)
height_scale = tk.Scale(frame, from_=100, to=250, orient=tk.HORIZONTAL, length=200, font=("Arial", 10), tickinterval=25)
height_scale.set(170)  # Default value
height_scale.grid(row=3, column=1, padx=10, pady=10)

# Create a button to calculate BMI
calculate_button = tk.Button(frame, text="Calculate BMI", font=("Arial", 12, "bold"), bg="#32CD32", fg="white", command=calculate_bmi)
calculate_button.grid(row=4, column=0, columnspan=2, pady=10)

# Create a button to reset fields
reset_button = tk.Button(frame, text="Reset", font=("Arial", 12, "bold"), bg="#FF6347", fg="white", command=reset_fields)
reset_button.grid(row=5, column=0, columnspan=2, pady=10)

# Create a button to save data
save_button = tk.Button(frame, text="Save Data", font=("Arial", 12, "bold"), bg="#1E90FF", fg="white", command=save_data)
save_button.grid(row=6, column=0, columnspan=2, pady=10)
save_button.config(state=tk.DISABLED)  # Disabled until BMI is calculated

# Create labels to display BMI, category, suggestion, and daily calorie needs
bmi_label = tk.Label(root, text="Your BMI: ", font=("Arial", 12), bg="#F5F5F5")
bmi_label.grid(row=3, column=0, pady=10)

category_label = tk.Label(root, text="Category: ", font=("Arial", 12), bg="#F5F5F5")
category_label.grid(row=4, column=5, pady=10)

suggestion_label = tk.Label(root, text="Suggestion: ", font=("Arial", 12), bg="#F5F5F5")
suggestion_label.grid(row=5, column=0, pady=10)

calories_label = tk.Label(root, text="Daily Calorie Needs: ", font=("Arial", 12), bg="#F5F5F5")
calories_label.grid(row=5, column=5, pady=10)

# Frame to hold the matplotlib plot
plot_frame = tk.Frame(root, bg="white")
plot_frame.grid(row=0, column=8, padx=20, pady=20)

# Run the application
root.mainloop()
